In [ ]:
#pip install folium
import folium

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
import urllib
from urllib import request
import urllib.request
import json
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
from sklearn import preprocessing

In [ ]:
#pip install geojson

In [ ]:
import os
import json
import random
import requests
import geojson

In [ ]:
import branca
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

In [ ]:
path = os.getcwd()
os.chdir("../")
# импорт функции rename_subj для приведения всех названий регионов к единому стандарту
from Utils.subj_rename import rename_subj
# импорт функции, сохраняющей карту в хорошем качестве в png формат
from Utils.save_map import map_to_png
os.chdir(path.split("\\")[-1])

In [ ]:
# импорт словаря переименований регионов для унифицирования их названия в используемых df
with open("../Utils/subj_rename.json", encoding="utf-8") as file:
    subj_rename_data = json.load(file)

In [ ]:
df_food = pd.read_excel("food.xls")

In [ ]:
df_food

In [ ]:
#df_food  = pd.read_excel("food.xlsx")
df_food["region"] = df_food["region"].str.strip()

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_food = rename_subj(
    df_food,
    "region",
    subj_rename_data
)
display(df_food.head())
df_food.shape

In [ ]:
#pip install openpyxl

In [ ]:
df_regions = gpd.read_file("../Utils/geo_data.json").rename(columns={"name": "region"})

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_regions = rename_subj(
    df_regions,
    "region",
    subj_rename_data
)
display(df_regions.head())
df_regions.shape

In [ ]:
df_show = df_regions.merge(df_food, on="region", how="right")
display(df_show.head())
df_show.shape

In [ ]:
type(df_show)

In [ ]:
type(df_show.drop(['food'], axis = 1))

In [ ]:
df_show = df_show.dropna()

In [ ]:
Map = folium.Map(
    [69, 96],
    zoom_start=3.5,
    dragging=True,
    zoom_control=True,
    scrollWheelZoom=True,
  )

Popup = GeoJsonPopup(
    fields=['region','food'],
    #aliases = ["Регион", "Данные"]
    localize=True,
    labels=True,  
)

folium.Choropleth(
    geo_data=df_show,
    data=df_show,
    columns=['region', 'food'],   
    legend_name='Оборот общественного питания на душу населения (в рублях)',
    key_on='feature.properties.region',
    fill_color="Reds",
    fill_opacity=0.8,
    line_opacity=0.5,
    highlight=True,
).add_to(Map)

folium.GeoJson(
    df_show,
    style_function=lambda feature: {
        'fillColor': '#ffffff',
        'color': 'ffffff',
        'weight': 1,
        'opacity': 0.00001,
        'line_opacity': 0.9,
        'line_weight': 1
    },
    highlight_function=lambda feature: {
        'fillColor': '#98ff98',
        'color': 'yellow',
        'weight': 1,
        'line_opacity': 1,
        'line_weight': 1,
        'opacity': 1
    },
    #tooltip=Tooltip,
    popup=Popup).add_to(Map)

#folium.LayerControl().add_to(Map)

tooltip = "Click Here For More Info"
# координаты середины канала - 66°25' с. ш. 94°15' в. д.
marker = folium.Marker(
    location=[66.25, 91.15],
    popup="""
        <stong>
            <a href='https://t.me/maps_stat'>tg: https://t.me/maps_stat</a><br />
            <a href='https://vk.com/maps_stat'>vk: https://vk.com/maps_stat</a>
        </stong>
    """,
    icon=folium.DivIcon(
        html="""
            <div style="color:#000000;background:#878787:90px;text-align:center;"><h5><u>@maps_stat</u><h5></div>
        """
    ),
    fill_opacity=0.5
)
marker.add_to(Map)

folium.LayerControl().add_to(Map)
Map

In [ ]:
# сохранение карты в html-формат
Map.save("food.html")

In [ ]:
# загрузка конфига
# нужен для корректной работы функции сохранения карты в виде изображения
with open("../config.json") as file:
    config = json.load(file)

In [ ]:
%%time
# Сохранение карты в виде изображения
map_to_png(Map, config, "food.png")